## Physics-Informed Neural Network for 1D Heat Equation in PyTorch

This notebook implements a Physics-Informed Neural Network (PINN) to solve the 1D heat equation using PyTorch.

### Problem Statement

We want to solve the 1D heat equation:

$$ \frac{\partial u}{\partial t}(x,t) = \lambda \frac{\partial^2 u}{\partial x^2}(x,t)$$

for $x\in [0,L]$ and $t>0$ with:

**Initial condition:**
$$ u(x,0) = \sin(\pi x/L) $$

**Boundary conditions:**
$$ u(0,t) = u(L,t) = 0$$

The physics-informed neural network approach minimizes the residual of the PDE:

$$ D_u(x,t) = \frac{\partial u}{\partial t} - \lambda \frac{\partial^2 u}{\partial x^2} = 0 $$

## Imports and Setup

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from pyDOE import lhs

# Set random seeds for reproducibility
np.random.seed(1234)
torch.manual_seed(1234)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## Problem Parameters

In [ ]:
# Domain parameters
L = 1.0          # Length of spatial domain
T_max = 1.0      # Maximum time
lam = 0.01       # Thermal diffusivity (lambda)

# Domain bounds
x_min, x_max = 0.0, L
t_min, t_max = 0.0, T_max

print(f'Spatial domain: [{x_min}, {x_max}]')
print(f'Time domain: [{t_min}, {t_max}]')
print(f'Thermal diffusivity: {lam}')

## Neural Network Architecture

In [ ]:
class PINN(nn.Module):
    """Physics-Informed Neural Network for the Heat Equation"""
    
    def __init__(self, layers):
        super(PINN, self).__init__()
        
        # Network architecture
        self.layers = layers
        self.num_layers = len(layers)
        
        # Create network layers
        self.linears = nn.ModuleList([
            nn.Linear(layers[i], layers[i+1]) for i in range(self.num_layers - 1)
        ])
        
        # Initialize weights using Xavier initialization
        for i in range(self.num_layers - 1):
            nn.init.xavier_normal_(self.linears[i].weight)
            nn.init.zeros_(self.linears[i].bias)
    
    def forward(self, x, t):
        """Forward pass through the network
        
        Args:
            x: spatial coordinates (batch_size, 1)
            t: temporal coordinates (batch_size, 1)
        
        Returns:
            u: predicted temperature (batch_size, 1)
        """
        # Concatenate inputs
        inputs = torch.cat([x, t], dim=1)
        
        # Forward pass through hidden layers with tanh activation
        h = inputs
        for i in range(self.num_layers - 2):
            h = torch.tanh(self.linears[i](h))
        
        # Output layer (linear)
        u = self.linears[-1](h)
        return u
    
    def pde_residual(self, x, t, lam):
        """Compute the PDE residual D_u = u_t - lambda * u_xx
        
        Args:
            x: spatial coordinates
            t: temporal coordinates
            lam: thermal diffusivity
        
        Returns:
            residual: PDE residual
        """
        # Enable gradient computation
        x.requires_grad_(True)
        t.requires_grad_(True)
        
        # Compute u
        u = self.forward(x, t)
        
        # First derivatives
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u_x = torch.autograd.grad(
            u, x,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        
        # Second derivative
        u_xx = torch.autograd.grad(
            u_x, x,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        # PDE residual: u_t - lambda * u_xx = 0
        residual = u_t - lam * u_xx
        
        return residual

# Define network architecture
layers = [2, 32, 32, 32, 1]  # [input_dim, hidden1, hidden2, hidden3, output_dim]
model = PINN(layers).to(device)

print(f'Network architecture: {layers}')
print(f'Total parameters: {sum(p.numel() for p in model.parameters())}')
print(model)

## Generate Training Data

In [ ]:
# Number of training points
N_ic = 100      # Initial condition points
N_bc = 100      # Boundary condition points
N_pde = 10000   # Collocation points for PDE residual

# Initial condition: u(x, 0) = sin(pi * x / L)
x_ic = np.random.uniform(x_min, x_max, (N_ic, 1))
t_ic = np.zeros((N_ic, 1))
u_ic = np.sin(np.pi * x_ic / L)

# Boundary condition at x = 0: u(0, t) = 0
x_bc1 = np.zeros((N_bc, 1))
t_bc1 = np.random.uniform(t_min, t_max, (N_bc, 1))
u_bc1 = np.zeros((N_bc, 1))

# Boundary condition at x = L: u(L, t) = 0
x_bc2 = L * np.ones((N_bc, 1))
t_bc2 = np.random.uniform(t_min, t_max, (N_bc, 1))
u_bc2 = np.zeros((N_bc, 1))

# Combine all initial and boundary conditions
x_data = np.vstack([x_ic, x_bc1, x_bc2])
t_data = np.vstack([t_ic, t_bc1, t_bc2])
u_data = np.vstack([u_ic, u_bc1, u_bc2])

# Collocation points for PDE residual (Latin Hypercube Sampling)
lb = np.array([x_min, t_min])
ub = np.array([x_max, t_max])
X_pde = lb + (ub - lb) * lhs(2, N_pde)
x_pde = X_pde[:, 0:1]
t_pde = X_pde[:, 1:2]

# Convert to PyTorch tensors
x_data_tensor = torch.tensor(x_data, dtype=torch.float32, requires_grad=False).to(device)
t_data_tensor = torch.tensor(t_data, dtype=torch.float32, requires_grad=False).to(device)
u_data_tensor = torch.tensor(u_data, dtype=torch.float32, requires_grad=False).to(device)

x_pde_tensor = torch.tensor(x_pde, dtype=torch.float32, requires_grad=True).to(device)
t_pde_tensor = torch.tensor(t_pde, dtype=torch.float32, requires_grad=True).to(device)

print(f'Initial condition points: {N_ic}')
print(f'Boundary condition points: {2 * N_bc}')
print(f'PDE collocation points: {N_pde}')
print(f'Total data points: {x_data.shape[0]}')

## Visualize Training Points

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.scatter(t_data, x_data, c='blue', s=10, alpha=0.6, label='IC/BC points')
plt.xlabel('t')
plt.ylabel('x')
plt.title('Initial and Boundary Condition Points')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(t_pde, x_pde, c='red', s=1, alpha=0.3, label='Collocation points')
plt.xlabel('t')
plt.ylabel('x')
plt.title('PDE Collocation Points')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Training

In [ ]:
# Training parameters
epochs = 10000
learning_rate = 1e-3

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Loss history
loss_history = []
loss_data_history = []
loss_pde_history = []

# Training loop
for epoch in range(epochs):
    # Zero gradients
    optimizer.zero_grad()
    
    # Predict u at data points
    u_pred = model(x_data_tensor, t_data_tensor)
    
    # Data loss (MSE for initial and boundary conditions)
    loss_data = torch.mean((u_pred - u_data_tensor) ** 2)
    
    # PDE residual loss
    residual = model.pde_residual(x_pde_tensor, t_pde_tensor, lam)
    loss_pde = torch.mean(residual ** 2)
    
    # Total loss
    loss = loss_data + loss_pde
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    # Store losses
    loss_history.append(loss.item())
    loss_data_history.append(loss_data.item())
    loss_pde_history.append(loss_pde.item())
    
    # Print progress
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}, '
              f'Data Loss: {loss_data.item():.6f}, PDE Loss: {loss_pde.item():.6f}')

print('\nTraining completed!')

## Visualize Training Loss

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.semilogy(loss_history, 'b-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Total Loss')
plt.title('Training Loss')
plt.grid(True)

plt.subplot(1, 3, 2)
plt.semilogy(loss_data_history, 'g-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Data Loss')
plt.title('Data Loss (IC/BC)')
plt.grid(True)

plt.subplot(1, 3, 3)
plt.semilogy(loss_pde_history, 'r-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('PDE Loss')
plt.title('PDE Residual Loss')
plt.grid(True)

plt.tight_layout()
plt.show()

## Prediction and Visualization

In [ ]:
# Create a grid for prediction
N_x = 256
N_t = 100

x_test = np.linspace(x_min, x_max, N_x)
t_test = np.linspace(t_min, t_max, N_t)
X_grid, T_grid = np.meshgrid(x_test, t_test)

x_flat = X_grid.flatten()[:, None]
t_flat = T_grid.flatten()[:, None]

# Convert to tensors
x_test_tensor = torch.tensor(x_flat, dtype=torch.float32).to(device)
t_test_tensor = torch.tensor(t_flat, dtype=torch.float32).to(device)

# Predict
model.eval()
with torch.no_grad():
    u_pred = model(x_test_tensor, t_test_tensor)

# Reshape for plotting
u_pred_grid = u_pred.cpu().numpy().reshape(N_t, N_x)

print('Prediction completed!')

## Analytical Solution for Comparison

The analytical solution for the heat equation with the given initial and boundary conditions is:

$$ u(x,t) = \sin(\pi x/L) \exp(-\lambda (\pi/L)^2 t) $$

In [ ]:
# Analytical solution
u_analytical = np.sin(np.pi * X_grid / L) * np.exp(-lam * (np.pi / L) ** 2 * T_grid)

# Compute error
error = np.linalg.norm(u_pred_grid - u_analytical, 2) / np.linalg.norm(u_analytical, 2)
print(f'Relative L2 error: {error:.6e}')

## Compare PINN Prediction with Analytical Solution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# PINN prediction
ax = axes[0, 0]
im1 = ax.imshow(u_pred_grid.T, interpolation='nearest', cmap='hot',
                extent=[t_min, t_max, x_min, x_max],
                origin='lower', aspect='auto', vmin=-1, vmax=1)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_title('PINN Prediction')
plt.colorbar(im1, ax=ax)

# Plot training data points
ax.scatter(t_data, x_data, c='cyan', s=2, alpha=0.5, label='Training points')
ax.legend(loc='upper right')

# Analytical solution
ax = axes[0, 1]
im2 = ax.imshow(u_analytical.T, interpolation='nearest', cmap='hot',
                extent=[t_min, t_max, x_min, x_max],
                origin='lower', aspect='auto', vmin=-1, vmax=1)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_title('Analytical Solution')
plt.colorbar(im2, ax=ax)

# Absolute error
ax = axes[1, 0]
error_abs = np.abs(u_pred_grid - u_analytical)
im3 = ax.imshow(error_abs.T, interpolation='nearest', cmap='viridis',
                extent=[t_min, t_max, x_min, x_max],
                origin='lower', aspect='auto')
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_title('Absolute Error')
plt.colorbar(im3, ax=ax)

# Temporal slices at different times
ax = axes[1, 1]
time_indices = [0, N_t // 4, N_t // 2, 3 * N_t // 4, N_t - 1]
colors = ['blue', 'green', 'orange', 'red', 'purple']

for idx, color in zip(time_indices, colors):
    t_val = t_test[idx]
    ax.plot(x_test, u_pred_grid[idx, :], '--', color=color, 
            linewidth=2, label=f'PINN t={t_val:.2f}')
    ax.plot(x_test, u_analytical[idx, :], '-', color=color, 
            linewidth=1, alpha=0.7, label=f'Exact t={t_val:.2f}')

ax.set_xlabel('x')
ax.set_ylabel('u(x, t)')
ax.set_title('Spatial Profiles at Different Times')
ax.legend(fontsize=8, ncol=2)
ax.grid(True)

plt.tight_layout()
plt.show()

print(f'\nRelative L2 error: {error:.6e}')
print(f'Maximum absolute error: {np.max(error_abs):.6e}')

## Visualize PDE Residual

In [ ]:
# Compute PDE residual on the test grid
x_test_tensor_grad = torch.tensor(x_flat, dtype=torch.float32, requires_grad=True).to(device)
t_test_tensor_grad = torch.tensor(t_flat, dtype=torch.float32, requires_grad=True).to(device)

residual_test = model.pde_residual(x_test_tensor_grad, t_test_tensor_grad, lam)
residual_grid = residual_test.detach().cpu().numpy().reshape(N_t, N_x)

plt.figure(figsize=(10, 6))
im = plt.imshow(residual_grid.T ** 2, interpolation='nearest', cmap='viridis',
                extent=[t_min, t_max, x_min, x_max],
                origin='lower', aspect='auto')
plt.colorbar(im, label='Squared Residual')
plt.xlabel('t')
plt.ylabel('x')
plt.title('PDE Residual² (Should be close to 0)')
plt.show()

print(f'Mean squared residual: {np.mean(residual_grid ** 2):.6e}')

## Create Animation of the Solution

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Create animation
fig, ax = plt.subplots(figsize=(10, 6))

line_pinn, = ax.plot([], [], 'b-', linewidth=2, label='PINN')
line_exact, = ax.plot([], [], 'r--', linewidth=2, label='Analytical')

ax.set_xlim(x_min, x_max)
ax.set_ylim(-1.1, 1.1)
ax.set_xlabel('x')
ax.set_ylabel('u(x, t)')
ax.set_title('1D Heat Equation Solution')
ax.legend()
ax.grid(True)

time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12,
                    verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

def init():
    line_pinn.set_data([], [])
    line_exact.set_data([], [])
    time_text.set_text('')
    return line_pinn, line_exact, time_text

def animate(frame):
    line_pinn.set_data(x_test, u_pred_grid[frame, :])
    line_exact.set_data(x_test, u_analytical[frame, :])
    time_text.set_text(f't = {t_test[frame]:.3f}')
    return line_pinn, line_exact, time_text

ani = FuncAnimation(fig, animate, init_func=init, frames=N_t, 
                   interval=50, blit=True, repeat=True)

plt.close()  # Prevent duplicate display
HTML(ani.to_jshtml())

## Summary

This notebook demonstrates how to solve the 1D heat equation using a Physics-Informed Neural Network (PINN) implemented in PyTorch. The key features are:

1. **Neural Network**: A fully connected network with tanh activation functions
2. **Physics-Informed Loss**: Combination of data loss (initial/boundary conditions) and PDE residual loss
3. **Automatic Differentiation**: PyTorch's autograd computes the required derivatives for the PDE
4. **Training**: Adam optimizer minimizes the combined loss
5. **Validation**: Comparison with analytical solution shows the accuracy of the PINN approach

The PINN successfully learns the solution to the heat equation without requiring dense sampling of the solution in the interior of the domain, demonstrating the power of incorporating physical laws directly into the learning process.